In [52]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

In [53]:
df = pd.read_csv(r'C:\Users\HP\Desktop\VAFSUD\merged_nifty50_data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values('Date', inplace=True)

In [54]:
selected_stocks = ['TCS', 'INFY', 'HCLTECH', 'WIPRO', 'TECHM', 'LT', 
                   'BHARTIARTL', 'ICICIBANK', 'HDFCBANK', 'AXISBANK', 
                   'KOTAKBANK', 'SBIN', 'INDUSINDBK', 'YESBANK', 'TATAMOTORS']

In [55]:
df = df[df['Symbol'].isin(selected_stocks)]

In [56]:
df['MA_10'] = df['Close'].rolling(window=10).mean()
df['MA_50'] = df['Close'].rolling(window=50).mean()
df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()

In [57]:
delta = df['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))

In [58]:
df.bfill(inplace=True)

In [59]:
features = ['Close', 'MA_10', 'MA_50', 'EMA_10', 'RSI']
data = df[features].values

In [60]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)


In [61]:
def create_sequences(data, seq_length):
    x, y = [], []
    for i in range(len(data) - seq_length):
        x.append(data[i:i + seq_length])
        y.append(data[i + seq_length, 0])
    return np.array(x), np.array(y)

In [42]:
seq_length = 60
x, y = create_sequences(scaled_data, seq_length)

In [43]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 5))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 5))

In [44]:
inputs = Input(shape=(x_train.shape[1], 5))

In [45]:
x = LSTM(50, return_sequences=True)(inputs)
x = Dropout(0.2)(x)
x = LSTM(50, return_sequences=False)(x)
x = Dropout(0.2)(x)
x = Dense(25)(x)
outputs = Dense(1)(x)

In [46]:
# Create model
model = Model(inputs=inputs, outputs=outputs)

In [47]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stopping])

Epoch 1/20
2706/3088 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - loss: 0.0049 

In [ ]:
model.save('model.keras')

In [ ]:
predictions = model.predict(x_test)

In [ ]:
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
print(f'MAE: {mae:.2f}')
print(f'MSE: {mse:.2f}')